In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
import os
import numpy as np
import cv2
import heapq
import matplotlib.pyplot as plt
from PIL import Image

# Enable debugging for CUDA errors
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load Pretrained MobileNetV2
model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.DEFAULT)

num_classes = 2  # Ensure this matches the trained model
model.classifier = nn.Sequential(
    nn.Dropout(0.2),
    nn.Linear(model.last_channel, num_classes)
)

# Load Model Safely
try:
    model.load_state_dict(torch.load("mobilenet_uav123_optimized.pth", map_location=device), strict=False)
    print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")
    exit()

model = model.to(device)
model.eval()

# Image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load Image from File
def load_image(image_path):
    try:
        image = Image.open(image_path).convert("RGB")
        return image, cv2.imread(image_path)
    except Exception as e:
        print(f"Error loading image: {e}")
        return None, None

# Predict Obstacles from Image
def detect_obstacles(image):
    image_tensor = transform(image).unsqueeze(0).to(device)
    output = model(image_tensor)
    _, predicted_class = torch.max(output, 1)
    return predicted_class.item()

# A* Pathfinding Algorithm
def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def a_star_search(grid, start, goal):
    rows, cols = len(grid), len(grid[0])
    open_set = [(0, start)]
    came_from = {}
    g_score = {start: 0}
    f_score = {start: heuristic(start, goal)}
    
    while open_set:
        _, current = heapq.heappop(open_set)
        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            return path[::-1]
        
        neighbors = [(current[0] + dx, current[1] + dy) for dx, dy in [(-1,0), (1,0), (0,-1), (0,1), (-1,-1), (-1,1), (1,-1), (1,1)]]
        for neighbor in neighbors:
            if 0 <= neighbor[0] < rows and 0 <= neighbor[1] < cols and grid[neighbor[0]][neighbor[1]] == 0:
                tentative_g_score = g_score[current] + 1
                if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g_score
                    f_score[neighbor] = tentative_g_score + heuristic(neighbor, goal)
                    heapq.heappush(open_set, (f_score[neighbor], neighbor))
    return []

# Generate Grid Based on Obstacles
def generate_navigation_grid(image_path):
    grid = np.zeros((10, 10))  # 10x10 grid
    image, frame = load_image(image_path)
    if image:
        obstacle = detect_obstacles(image)
        grid[5][5] = 1 if obstacle else 0  # Example obstacle placement
    return grid, frame

# Compute Best Path for UAV and Provide Directions
def compute_uav_path(image_path):
    grid, frame = generate_navigation_grid(image_path)
    start = (5, 5)  # Assume UAV is at center
    goal_options = [(9, 5), (5, 9), (0, 5), (5, 0)]  # Possible forward directions
    
    best_path = None
    best_goal = None
    for goal in goal_options:
        path = a_star_search(grid, start, goal)
        if path:
            best_path = path
            best_goal = goal
            break  # Choose first available clear path
    
    if best_path:
        print(f"Optimal Path to {best_goal}: {best_path}")
        
        # Overlay path on image
        if frame is not None:
            for i in range(len(best_path) - 1):
                cv2.line(frame, (best_path[i][1] * 50 + 25, best_path[i][0] * 50 + 25),
                         (best_path[i+1][1] * 50 + 25, best_path[i+1][0] * 50 + 25), (0, 255, 0), 2)
            for (x, y) in best_path:
                cv2.circle(frame, (y * 50 + 25, x * 50 + 25), 5, (0, 255, 0), -1)
            cv2.imshow("UAV Navigation Path", frame)
            cv2.waitKey(1)
            cv2.destroyAllWindows()
        
        # Provide movement direction
        if best_goal[0] > start[0]:
            direction = "Move Forward"
        elif best_goal[0] < start[0]:
            direction = "Move Backward"
        elif best_goal[1] > start[1]:
            direction = "Move Right"
        else:
            direction = "Move Left"
        print("Recommended Direction:", direction)
        return direction
    else:
        print("No clear path available! Stopping.")
        return "Stop"

# Run Pathfinding with Local Image
image_path = "uav_image.jpg"  # Change to your image file path
compute_uav_path(image_path)


Using device: cuda
Model loaded successfully.
Optimal Path to (9, 5): [(6, 5), (7, 5), (8, 5), (9, 5)]
Recommended Direction: Move Forward


'Move Forward'